In [1]:
import pandas as pd
import pysam

In [2]:
hg38 = pysam.FastaFile("/databank/igenomes/Homo_sapiens/UCSC/hg38/Sequence/WholeGenomeFasta/genome.fa")

In [10]:
df

,chr,base_hg38,base+1,Allele_1,Allele_2,Proxy,POS,ID,REF,ALT,#CHROM,QUAL,FILTER,INFO
0,chr1,2217853,2217854,C,T,rs2503716,2217853,.,C,T,chr1,.,.,OLD_ID=rs2503716
1,chr1,2226509,2226510,C,T,rs1692580,2226509,.,T,C,chr1,.,.,OLD_ID=rs1692580
2,chr1,2231505,2231506,C,T,rs1692578,2231505,.,C,T,chr1,.,.,OLD_ID=rs1692578
3,chr1,2232129,2232130,C,T,rs1713705,2232129,.,C,T,chr1,.,.,OLD_ID=rs1713705
4,chr1,2232677,2232678,T,C,rs1692579,2232677,.,T,C,chr1,.,.,OLD_ID=rs1692579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21954,chr9,136352316,136352317,G,A,rs61386106,136352316,.,G,A,chr9,.,.,OLD_ID=rs61386106
21955,chr9,136353630,136353631,A,G,rs28642213,136353630,.,A,G,chr9,.,.,OLD_ID=rs28642213
21956,chr9,137751669,137751670,A,G,rs11792069,137751669,.,A,G,chr9,.,.,OLD_ID=rs11792069
21957,chr9,137753559,137753560,G,T,rs35528310,137753559,.,G,T,chr9,.,.,OLD_ID=rs35528310


In [7]:
df = pd.read_csv("CAD_SNP_MLV.txt", sep="\t")
df = df[["chr", "base_hg38", "base+1", "Allele_1", "Allele_2", "Proxy"]]

In [9]:
matching = {False: 0, True: 0}
first_allele = {False: 0, True: 0}

positions = []
refs = []
alts = []

for _, row in df.iterrows():
    if "-" not in row.Allele_1 + row.Allele_2:
        bp =  hg38.fetch(row.chr, row.base_hg38 - 1, row.base_hg38 + len(row.Allele_1) - 1).upper()
        positions.append(row.base_hg38)
        refs.append(bp)
        if bp == row.Allele_1:  
            alts.append(row.Allele_2)
        elif bp == row.Allele_2:
            alts.append(row.Allele_1)
        else:
            alts.append("{},{}".format(row.Allele_1, row.Allele_2))
    elif row.Allele_2 == "-":
        bp = hg38.fetch(row.chr, row.base_hg38 - 1, row.base_hg38 + len(row.Allele_1)).upper()
        positions.append(row.base_hg38)
        refs.append(bp)
        alts.append(bp[0])
        if bp[1:] != row.Allele_1:
            print(row.Allele_1, row.Allele_2, bp)
    else:
        bp = hg38.fetch(row.chr, row.base_hg38 - 1, row.base_hg38).upper()
        positions.append(row.base_hg38)
        refs.append(bp)
        alts.append(row.Allele_2)

df["POS"] = positions
df["ID"] = "." # df.Proxy
df["REF"] = refs
df["ALT"] = alts
df["#CHROM"] = df.chr
df["QUAL"] = "."
df["FILTER"] = "."
df["INFO"] = "OLD_ID=" + df.Proxy

In [19]:
with open("CAD_SNP_MLV.vcf", "w") as fp:
    fp.write("##fileformat=VCFv4.2\n")
    fp.write("##INFO=<ID=OLD_ID,Number=A,Type=String,Description=\"The old rsID\">\n")

for _, row in df.iterrows():
    assert hg38.fetch(
        row["#CHROM"], row["POS"] - 1, row["POS"] + len(row.REF) - 1
    ).upper() == row.REF

df = df[["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]]
df.to_csv("CAD_SNP_MLV.vcf", index=False, sep="\t", mode='a')

In [33]:
import pysam

dbSNP = pysam.VariantFile(
    "/project/Wellcome_Discovery/shared/00_publicly_available/06_dbSNP/dbSNP.v156.hg38.vcf.gz"
)
new_rsids = []

for _, row in df.iterrows():
    variants = list(dbSNP.fetch(row.chr, row.POS - 1, row.POS))
    variants = [
        v for v in variants if row.REF == v.ref and row.ALT in v.alts
    ]
    if len(variants) > 1:
        print(variants[0].chrom, variants[0].pos, variants[0].id, variants[0].ref, variants[0].alts)
        print(variants[0].chrom, variants[1].pos, variants[1].id, variants[1].ref, variants[1].alts)
    
    if len(variants) == 0:
        new_rsids.append(".")
    elif len(variants) == 1:
        new_rsids.append(variants[0].id)
    else:
        new_rsids.append(",".join([v.id for v in variants]))

print(new_rsids)

chr11 13262101 rs1564876101 G ('GTTTT',)
chr11 13262101 rs2133399259 G ('GTTTT',)
chr11 119047102 rs501351 T ('G',)
chr11 119047102 rs2133555081 T ('C', 'G')
chr11 119057538 rs11217133 G ('A', 'C')
chr11 119057538 rs2133621996 G ('A',)
chr11 119057542 rs2509121 C ('T',)
chr11 119057542 rs2133622010 C ('T',)
chr11 119057542 rs2509121 C ('T',)
chr11 119057542 rs2133622010 C ('T',)
chr11 119060319 rs11217137 A ('G',)
chr11 119060319 rs2133629628 A ('G',)
chr11 119064719 rs59713308 TAAAATA ('T',)
chr11 119064719 rs2133632360 TAAAATA ('T', 'TAAAATAAAAATA')
chr11 119066395 rs550974 A ('C', 'G', 'T')
chr11 119066395 rs2133634264 A ('T',)
chr11 119066439 rs622478 G ('C',)
chr11 119066439 rs2133634322 G ('C',)
chr11 119066733 rs4454730 G ('A',)
chr11 119066733 rs2133635669 G ('A',)
chr11 119066807 rs4545564 T ('C',)
chr11 119066807 rs2133635900 T ('C',)
chr11 119067274 rs1784461 G ('A',)
chr11 119067274 rs2133637108 G ('A',)
chr11 119067604 rs1786141 C ('A', 'G', 'T')
chr11 119067604 rs21336378

In [35]:
df["ID"] = new_rsids

In [39]:
df.to_csv(
    "CAD_SNP_MLV.dbSNP.v156.txt", index=False, sep="\t"
)

Counter({True: 14394, False: 7565})